In [1]:
#!pip install pymannkendall
%matplotlib inline
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU,SimpleRNN, Dense, Bidirectional,Dropout, TimeDistributed
from sklearn.model_selection import train_test_split,KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.optimizers import Adam, RMSprop,SGD
from sklearn.metrics import mean_squared_error, mean_absolute_error, median_absolute_error
from scipy.stats import gmean

In [ ]:
# Load your time series data
time_series_data = pd.read_csv(r"C:\Users\edmun\Desktop\UTRGV Thesis\ARIMA_ETS\Data and Codes\USA.csv")
time_series_data['Date'] = pd.to_datetime(time_series_data['Date'])
time_series_data.set_index('Date', inplace=True)

# Display the first few rows of the dataset to understand its structure
time_series_data.tail()

## Data Preparation

In [5]:
# Prepare data for RNN
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(time_series_data['FluAcases'].values.reshape(-1, 1))

train_size = int(len(scaled_data) * 0.933)
train_data = scaled_data[:train_size]
test_data = scaled_data[train_size:]

### You can also use this for both train and test data
# Train: Jan 2009 - Dec 2022
train_data = scaled_data['2009-01-01':'2022-12-01']

# Test: Jan 2023 - Dec 2023
test_data = scaled_data['2023-01-01':'2023-12-01']

print(f"Train shape: {train_data.shape}, Test shape: {test_data.shape}")

def create_dataset(dataset, time_step=1):
    X, Y = [], []
    for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step), 0]
        X.append(a)
        Y.append(dataset[i + time_step, 0])
    return np.array(X), np.array(Y)

time_step = 12
X_train, y_train = create_dataset(train_data, time_step)
X_test, y_test = create_dataset(test_data, time_step)

X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

## Long Short Term Memory (LSTM)

In [ ]:
units = [16, 32, 64, 128, 256, 512]  # Number of units in LSTM
activation_functions = ["relu", "tanh", "sigmoid"]  # Activation functions
learning_rates = [0.001, 0.01,0.1]  # Learning rates
optimizers = ["adam", "rmsprop", "sgd"]  # Optimizers
batch_sizes = [16, 32, 64,128]  # Batch sizes

best_score = float("inf")
best_params = {}

for unit in units:
    for activation in activation_functions:
        for lr in learning_rates:
            for opt in optimizers:
                for batch_size in batch_sizes:
                    model = Sequential()
                    model.add(LSTM(units=unit, activation=activation, input_shape=(time_step, 1), return_sequences=True))
                    model.add(Dropout(0.3))
                    model.add(LSTM(units=unit, return_sequences=True))
                    model.add(Dropout(0.3))
                    model.add(TimeDistributed(Dense(1)))
                    
                    if opt == "adam":
                        optimizer = Adam(learning_rate=lr)
                    elif opt == "rmsprop":
                        optimizer = RMSprop(learning_rate=lr)
                    elif opt == "sgd":
                        optimizer = SGD(learning_rate=lr)
                    
                    model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mean_squared_error'])
                    
                    history = model.fit(X_train, y_train, epochs=5, batch_size=batch_size, validation_split=0.2, shuffle=False, verbose=1)
                    
                    val_mse = min(history.history['val_mean_squared_error'])
                    
                    if val_mse < best_score:
                        best_score = val_mse
                        best_params = {
                            "units": unit,
                            "activation": activation,
                            "learning_rate": lr,
                            "optimizer": opt,
                            "batch_size": batch_size
                        }

print("Best parameters found: ", best_params)

print("Best validation MSE: ", best_score)

## Gated Recurrent Neural Network (GRU)

In [ ]:
units = [16, 32, 64, 128, 256, 512]  # Number of units in GRU
activation_functions = ["relu", "tanh", "sigmoid"]  # Activation functions
learning_rates = [0.001, 0.01,0.1]  # Learning rates
optimizers = ["adam", "rmsprop", "sgd"]  # Optimizers
batch_sizes = [16, 32, 64,128]  # Batch sizes

best_score = float("inf")
best_params = {}

for unit in units:
    for activation in activation_functions:
        for lr in learning_rates:
            for opt in optimizers:
                for batch_size in batch_sizes:
                    model = Sequential()
                    model.add(GRU(units=unit, activation=activation, input_shape=(time_step, 1), return_sequences=True))
                    model.add(Dropout(0.5))
                    model.add(GRU(units=unit, return_sequences=True))
                    model.add(Dropout(0.5))
                    model.add(TimeDistributed(Dense(1)))
                    
                    if opt == "adam":
                        optimizer = Adam(learning_rate=lr)
                    elif opt == "rmsprop":
                        optimizer = RMSprop(learning_rate=lr)
                    elif opt == "sgd":
                        optimizer = SGD(learning_rate=lr)
                    
                    model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mean_squared_error'])
                    
                    history = model.fit(X_train, y_train, epochs=5, batch_size=batch_size, validation_split=0.2, shuffle=False, verbose=1)
                    
                    val_mse = min(history.history['val_mean_squared_error'])
                    
                    if val_mse < best_score:
                        best_score = val_mse
                        best_params = {
                            "units": unit,
                            "activation": activation,
                            "learning_rate": lr,
                            "optimizer": opt,
                            "batch_size": batch_size
                        }

print("Best parameters found: ", best_params)
print("Best validation MSE: ", best_score)

## Simple Recurrent Neural Network (SimpleRNN)

In [ ]:
units = [16, 32, 64, 128, 256, 512] # Number of units in SimpleRnn
activation_functions = ["relu", "tanh", "sigmoid"]  # Activation functions
learning_rates = [0.001, 0.01,0.1]  # Learning rates
optimizers = ["adam", "rmsprop", "sgd"]  # Optimizers
batch_sizes = [16, 32, 64,128]  # Batch sizes

best_score = float("inf")
best_params = {}

for unit in units:
    for activation in activation_functions:
        for lr in learning_rates:
            for opt in optimizers:
                for batch_size in batch_sizes:
                    model = Sequential()
                    model.add(SimpleRNN(units=unit, activation=activation, input_shape=(time_step, 1), return_sequences=True))
                    model.add(Dropout(0.3))
                    model.add(SimpleRNN(units=unit, return_sequences=True))
                    model.add(Dropout(0.3))
                    model.add(TimeDistributed(Dense(1)))
                    
                    if opt == "adam":
                        optimizer = Adam(learning_rate=lr)
                    elif opt == "rmsprop":
                        optimizer = RMSprop(learning_rate=lr)
                    elif opt == "sgd":
                        optimizer = SGD(learning_rate=lr)
                    
                    model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mean_squared_error'])
                    
                    history = model.fit(X_train, y_train, epochs=5, batch_size=batch_size, validation_split=0.2, shuffle=False, verbose=1)
                    
                    val_mse = min(history.history['val_mean_squared_error'])
                    
                    if val_mse < best_score:
                        best_score = val_mse
                        best_params = {
                            "units": unit,
                            "activation": activation,
                            "learning_rate": lr,
                            "optimizer": opt,
                            "batch_size": batch_size
                        }

print("Best parameters found: ", best_params)
print("Best validation MSE: ", best_score)

### BiLSTM

In [ ]:
from keras.models import Sequential
from keras.layers import Bidirectional, LSTM, Dropout, TimeDistributed, Dense
from keras.optimizers import Adam, RMSprop, SGD

units = [16, 32, 64, 128, 256, 512]  # Number of units in LSTM
activation_functions = ["relu", "tanh", "sigmoid"]  # Activation functions (though 'tanh' is standard for LSTM)
learning_rates = [0.001, 0.01, 0.1]  # Learning rates
optimizers = ["adam", "rmsprop", "sgd"]  # Optimizers
batch_sizes = [16, 32, 64, 128]  # Batch sizes

best_score = float("inf")
best_params = {}

for unit in units:
    for activation in activation_functions:
        for lr in learning_rates:
            for opt in optimizers:
                for batch_size in batch_sizes:
                    # Construct the model
                    model = Sequential()
                    model.add(Bidirectional(LSTM(units=unit, activation=activation, return_sequences=True), input_shape=(time_step, 1)))
                    model.add(Dropout(0.3))
                    model.add(Bidirectional(LSTM(units=unit, activation=activation, return_sequences=True)))
                    model.add(Dropout(0.3))
                    model.add(TimeDistributed(Dense(1)))
                    
                    # Pick optimizer based on loop
                    if opt == "adam":
                        optimizer = Adam(learning_rate=lr)
                    elif opt == "rmsprop":
                        optimizer = RMSprop(learning_rate=lr)
                    elif opt == "sgd":
                        optimizer = SGD(learning_rate=lr)
                    
                    model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mean_squared_error'])
                    
                    # Train the model briefly
                    history = model.fit(
                        X_train, y_train,
                        epochs=5,
                        batch_size=batch_size,
                        validation_split=0.2,
                        shuffle=False,
                        verbose=1
                    )
                    
                    # Grab the best validation score
                    val_mse = min(history.history['val_mean_squared_error'])
                    
                    # Save best params if this is the new best
                    if val_mse < best_score:
                        best_score = val_mse
                        best_params = {
                            "units": unit,
                            "activation": activation,
                            "learning_rate": lr,
                            "optimizer": opt,
                            "batch_size": batch_size
                        }

print("Best parameters found:", best_params)
print("Best validation MSE:", best_score)

### BiGRU

In [ ]:
from keras.models import Sequential
from keras.layers import Bidirectional, GRU, Dropout, TimeDistributed, Dense
from keras.optimizers import Adam, RMSprop, SGD

units = [16, 32, 64, 128, 256, 512]  # Number of units in GRU
activation_functions = ["relu", "tanh", "sigmoid"]  # GRUs usually use 'tanh', but you can play with these
learning_rates = [0.001, 0.01, 0.1]  # Learning rates
optimizers = ["adam", "rmsprop", "sgd"]  # Optimizers
batch_sizes = [16, 32, 64, 128]  # Batch sizes

best_score = float("inf")
best_params = {}

for unit in units:
    for activation in activation_functions:
        for lr in learning_rates:
            for opt in optimizers:
                for batch_size in batch_sizes:
                    model = Sequential()
                    model.add(
                        Bidirectional(
                            GRU(units=unit, activation=activation, return_sequences=True),
                            input_shape=(time_step, 1)
                        )
                    )
                    model.add(Dropout(0.3))
                    model.add(
                        Bidirectional(
                            GRU(units=unit, activation=activation, return_sequences=True)
                        )
                    )
                    model.add(Dropout(0.3))
                    model.add(TimeDistributed(Dense(1)))

                    # Set up optimizer
                    if opt == "adam":
                        optimizer = Adam(learning_rate=lr)
                    elif opt == "rmsprop":
                        optimizer = RMSprop(learning_rate=lr)
                    elif opt == "sgd":
                        optimizer = SGD(learning_rate=lr)

                    model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mean_squared_error'])

                    # Fit model, just 5 epochs for quick grid search
                    history = model.fit(
                        X_train, y_train,
                        epochs=5,
                        batch_size=batch_size,
                        validation_split=0.2,
                        shuffle=False,
                        verbose=1
                    )

                    val_mse = min(history.history['val_mean_squared_error'])

                    # Save best result
                    if val_mse < best_score:
                        best_score = val_mse
                        best_params = {
                            "units": unit,
                            "activation": activation,
                            "learning_rate": lr,
                            "optimizer": opt,
                            "batch_size": batch_size
                        }

print("Best parameters found:", best_params)
print("Best validation MSE:", best_score)

### Model Building

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
# Define Scaled Dot-Product Attention
class ScaledDotProductAttention(nn.Module):
    def __init__(self):
        super(ScaledDotProductAttention, self).__init__()

    def forward(self, query, key, value, mask=None):
        d_k = query.size(-1)
        scores = torch.matmul(query, key.transpose(-2, -1)) / torch.sqrt(torch.tensor(d_k, dtype=torch.float32))

        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9)

        attn = torch.nn.functional.softmax(scores, dim=-1)
        output = torch.matmul(attn, value)
        return output, attn

In [ ]:
# Define Multi-Head Attention
class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, d_model):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model
        self.d_k = d_model // num_heads

        self.query_linear = nn.Linear(d_model, d_model)
        self.key_linear = nn.Linear(d_model, d_model)
        self.value_linear = nn.Linear(d_model, d_model)
        self.out_linear = nn.Linear(d_model, d_model)
        self.attention = ScaledDotProductAttention()

    def forward(self, query, key, value, mask=None):
        batch_size = query.size(0)

        # Linear projections
        query = self.query_linear(query).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        key = self.key_linear(key).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        value = self.value_linear(value).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)

        # Apply attention
        attn_output, attn = self.attention(query, key, value, mask)

        # Concatenate and apply final linear layer
        attn_output = attn_output.transpose(1, 2).contiguous().view(batch_size, -1, self.d_model)
        output = self.out_linear(attn_output)

        return output

In [ ]:
# Feed Forward Network
class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super(FeedForward, self).__init__()
        self.linear1 = nn.Linear(d_model, d_ff)
        self.linear2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.linear2(self.relu(self.linear1(x)))

In [ ]:
# Define Transformer Encoder Layer
class TransformerEncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout=0.3):
        super(TransformerEncoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(num_heads, d_model)
        self.feed_forward = FeedForward(d_model, d_ff)

        self.layer_norm1 = nn.LayerNorm(d_model)
        self.layer_norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src, mask=None):
        # Self-attention and Add & Norm
        src2 = self.self_attn(src, src, src, mask)
        src = self.layer_norm1(src + self.dropout(src2))

        # Feed Forward and Add & Norm
        src2 = self.feed_forward(src)
        src = self.layer_norm2(src + self.dropout(src2))

        return src

In [ ]:
# Transformer Encoder
class TransformerEncoder(nn.Module):
    def __init__(self, num_layers, d_model, num_heads, d_ff, vocab_size, max_seq_len, dropout=0.3):
        super(TransformerEncoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_embedding = nn.Parameter(torch.zeros(1, max_seq_len, d_model))

        self.layers = nn.ModuleList([
            TransformerEncoderLayer(d_model, num_heads, d_ff, dropout)
            for _ in range(num_layers)
        ])
        self.dropout = nn.Dropout(dropout)

    def forward(self, src, mask=None):
        seq_len = src.size(1)
        src = self.embedding(src) + self.pos_embedding[:, :seq_len, :]
        src = self.dropout(src)

        for layer in self.layers:
            src = layer(src, mask)

        return src

In [ ]:
import pandas as pd
import numpy as np
import torch
from sklearn.preprocessing import MinMaxScaler

# Function to load and preprocess the data
def load_and_preprocess_data(file_path):
    # Load the data from CSV file
    Influenza_data = pd.read_csv(file_path)

    # Convert 'Start Date' to datetime and set as index
    Influenza_data['Date'] = pd.to_datetime(Influenza_data['Date'])
    Influenza_data.set_index('Date', inplace=True)

    # Select the 'Percent Expected COVID-19 Deaths' and fill missing values
    death_percentage = Influenza_data['FluAcases'].fillna(0)

    # Scale the data between 0 and 1
    scaler = MinMaxScaler()
    death_percentage_scaled = scaler.fit_transform(death_percentage.values.reshape(-1, 1))

    return death_percentage_scaled, scaler

In [ ]:
# Function to create sequences for time-series forecasting
def create_sequences(data, seq_length):
    sequences = []
    for i in range(len(data) - seq_length):
        sequences.append((data[i:i+seq_length], data[i+seq_length]))
    return sequences

In [ ]:
# Function to convert sequences to PyTorch tensors
def to_tensor(data):
    # Convert list of sequences into a single numpy array
    X_np = np.array([seq[0] for seq in data])
    y_np = np.array([seq[1] for seq in data])

    # Convert numpy arrays to PyTorch tensors
    X = torch.tensor(X_np, dtype=torch.float32)
    y = torch.tensor(y_np, dtype=torch.float32)

    return X, y

In [ ]:
# Main function to load data, preprocess, and create sequences
def prepare_data_for_transformer(file_path, seq_length=12):
    # Load and preprocess the data
    data_scaled, scaler = load_and_preprocess_data(file_path)

    # Create sequences
    sequences = create_sequences(data_scaled, seq_length)

    # Split into training and test sets (80% train, 20% test)
    train_size = int(len(sequences) * 0.933)
    train_sequences = sequences[:train_size]
    test_sequences = sequences[train_size:]

    # Convert to tensors
    X_train, y_train = to_tensor(train_sequences)
    X_test, y_test = to_tensor(test_sequences)

    return X_train, y_train, X_test, y_test, scaler

In [ ]:
# Example usage:
file_path = r"C:\Users\edmun\Desktop\UTRGV Thesis\TS Manuscript Prof - Copy\Data and Codes\USA.csv" # Update with your file path
X_train, y_train, X_test, y_test, scaler = prepare_data_for_transformer(file_path)

# Check the shapes of the tensors
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
# Adjusted Transformer Encoder for time-series forecasting
class TransformerTimeSeries(nn.Module):
    def __init__(self, num_layers, d_model, num_heads, d_ff, seq_len, dropout=0.3):
        super(TransformerTimeSeries, self).__init__()
        self.input_linear = nn.Linear(1, d_model)  # From 1D input to d_model dimension
        self.pos_embedding = nn.Parameter(torch.zeros(1, seq_len, d_model))  # Positional encoding

        self.layers = nn.ModuleList([
            TransformerEncoderLayer(d_model, num_heads, d_ff, dropout)
            for _ in range(num_layers)
        ])
        self.fc_out = nn.Linear(d_model, 1)  # Final output to predict next time step value
        self.dropout = nn.Dropout(dropout)

    def forward(self, src, mask=None):
        # src: (batch_size, seq_len, input_dim)
        src = self.input_linear(src)  # Linear transformation
        seq_len = src.size(1)
        src = src + self.pos_embedding[:, :seq_len, :]  # Add positional encoding
        src = self.dropout(src)

        for layer in self.layers:
            src = layer(src, mask)

        # Predict the next time step value
        output = self.fc_out(src[:, -1, :])  # Only take the last time step for prediction

        return output

In [ ]:
# Function to train the transformer model
def train_transformer(model, train_loader, num_epochs=50, lr=0.001):
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0

        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()

            # Forward pass
            output = model(X_batch)

            # Calculate loss
            loss = criterion(output, y_batch)
            loss.backward()

            # Update weights
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss / len(train_loader):.6f}")

# Preparing COVID-19 data for training
def prepare_dataloader(X_train, y_train, batch_size=32):
    train_dataset = TensorDataset(X_train, y_train)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    return train_loader


if __name__ == "__main__":
    # Assuming X_train, y_train are already loaded and preprocessed (from the steps you provided earlier)
    seq_len = X_train.shape[1]  # e.g., 4 (the sequence length)
    d_model = 64  # Embedding dimension
    num_heads = 3
    num_layers = 4
    d_ff = 128  # Feed-forward hidden layer size
    dropout = 0.3
    num_epochs = 50

    # Initialize the model
    model = TransformerTimeSeries(num_layers, d_model, num_heads, d_ff, seq_len, dropout)

    # Prepare the dataloader
    train_loader = prepare_dataloader(X_train, y_train, batch_size=32)

    # Train the model
    train_transformer(model, train_loader, num_epochs=num_epochs, lr=0.001)

In [ ]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt

# Function to make predictions on the training and testing sets
def predict(model, X_data):
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():  # No need to calculate gradients during inference
        predictions = model(X_data)
    return predictions

In [ ]:
# Function to evaluate model performance on training and testing sets
def evaluate_model(model, X_train, y_train, X_test, y_test):
    criterion = nn.MSELoss()

    # Predictions
    train_predictions = predict(model, X_train)
    test_predictions = predict(model, X_test)

    # Calculate loss
    train_loss = criterion(train_predictions, y_train).item()
    test_loss = criterion(test_predictions, y_test).item()

    print(f'Training Loss (MSE): {train_loss:.6f}')
    print(f'Test Loss (MSE): {test_loss:.6f}')

    return train_predictions, test_predictions

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

def plot_predictions(train_predictions, y_train, test_predictions, y_test, scaler):
    # Generate date range for the data points
    start_date = pd.Timestamp('2023-01-01')
    end_date = pd.Timestamp('2023-12-01')
    dates = pd.date_range(start=start_date, end=end_date, periods=len(y_train) + len(y_test))

    # Convert predictions and true values back to the original scale
    train_predictions_rescaled = scaler.inverse_transform(train_predictions.detach().numpy())
    y_train_rescaled = scaler.inverse_transform(y_train.detach().numpy())
    test_predictions_rescaled = scaler.inverse_transform(test_predictions.detach().numpy())
    y_test_rescaled = scaler.inverse_transform(y_test.detach().numpy())

    # Plot the results
    plt.figure(figsize=(12, 6))

    # Plot training data
    plt.plot(dates[:len(y_train_rescaled)], y_train_rescaled, label='Actual Training Data', color='blue')
    plt.plot(dates[:len(train_predictions_rescaled)], train_predictions_rescaled, label='Predicted Training Data', color='orange', linestyle='--')

    # Plot testing data
    plt.plot(dates[len(y_train_rescaled):len(y_train_rescaled) + len(y_test_rescaled)], y_test_rescaled, label='Actual Test Data', color='green')
    plt.plot(dates[len(y_train_rescaled):len(y_train_rescaled) + len(test_predictions_rescaled)], test_predictions_rescaled, label='Predicted Test Data', color='red', linestyle='--')

    plt.legend()
    plt.xlabel('Date')
    plt.ylabel('Number of Influenza A Cases')
    plt.show()

# Example usage of evaluation and plotting functions
if __name__ == "__main__":
    # Assume you have already trained the model and you have X_train, y_train, X_test, y_test, and the scaler
    train_predictions, test_predictions = evaluate_model(model, X_train, y_train, X_test, y_test)

    # Plot the predictions on top of actual data
    plot_predictions(train_predictions, y_train, test_predictions, y_test, scaler)

In [ ]:
import torch

def calculate_metrics(predictions, actuals):
    # Ensure the predictions and actuals are float tensors
    predictions = predictions.float()
    actuals = actuals.float()

    # MAE
    mae = torch.mean(torch.abs(actuals - predictions))
    
    # MSE
    mse = torch.mean((actuals - predictions) ** 2)

    # GMRAE
    gmrae = torch.exp(torch.mean(torch.abs(torch.log1p(actuals) - torch.log1p(predictions))))

    # Theil's U1 statistic
    errors = actuals - predictions
    errors_squared = errors ** 2
    numerator = torch.sqrt(torch.mean(errors_squared))
    denominator = torch.sqrt(torch.mean(actuals ** 2)) + torch.sqrt(torch.mean(predictions ** 2))
    theil_u1 = numerator / denominator
    
    # RMSE
    rmse = torch.sqrt(torch.mean((actuals - predictions) ** 2))
    
    # MAPE
    mape = torch.mean(torch.abs((actuals - predictions) / (actuals + 1e-8))) * 100  # Added epsilon to avoid division by zero
    
    # SMAPE
    smape = 100 * torch.mean(2 * torch.abs(actuals - predictions) / (torch.abs(actuals) + torch.abs(predictions) + 1e-8))

    return mae, mse, gmrae, theil_u1, rmse, mape, smape
    

# Assuming train_predictions, y_train, test_predictions, y_test are PyTorch tensors
train_metrics = calculate_metrics(train_predictions, y_train)
test_metrics = calculate_metrics(test_predictions, y_test)

print(f"Training Metrics: MAE: {train_metrics[0].item():.4f}, MSE: {train_metrics[1].item():.4f}, GMRAE: {train_metrics[2].item():.4f}, Theil's U1: {train_metrics[3].item():.4f}, RMSE: {train_metrics[4].item():.4f}, MAPE: {train_metrics[5].item():.2f}%, SMAPE: {train_metrics[6].item():.2f}%")
print(f"Testing Metrics: MAE: {test_metrics[0].item():.4f}, MSE: {test_metrics[1].item():.4f}, GMRAE: {test_metrics[2].item():.4f}, Theil's U1: {test_metrics[3].item():.4f}, RMSE: {test_metrics[4].item():.4f}, MAPE: {test_metrics[5].item():.2f}%, SMAPE: {test_metrics[6].item():.2f}%")

In [ ]:
# Example usage:
file_path = r"C:\Users\edmun\Desktop\UTRGV Thesis\TS Manuscript Prof - Copy\Data and Codes\USA.csv"  # Update with your file path
X_train, y_train, X_test, y_test, scaler = prepare_data_for_transformer(file_path)

# Check the shapes of the tensors
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Function to create the LSTM model
class LSTMTimeSeries(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim=1):
        super(LSTMTimeSeries, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))
        out = self.fc(out[:, -1, :]) 
        return out

# Adjust this function to train the model
def train_lstm(model, train_loader, num_epochs=50, lr=0.01):
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            output = model(X_batch)
            loss = criterion(output, y_batch)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss / len(train_loader):.6f}")

# Example of creating and training the model
if __name__ == "__main__":
    input_dim = 1  # number of features
    hidden_dim = 64  # number of features in hidden state
    num_layers = 4  # number of stacked lstm layers
    output_dim = 1  # number of output classes
    d_ff = 128  # Feed-forward hidden layer size
    dropout = 0.3
    num_epochs = 50

    model = LSTMTimeSeries(input_dim, hidden_dim, num_layers, output_dim)
    train_loader = prepare_dataloader(X_train, y_train, batch_size=32)
    train_lstm(model, train_loader)
### Repeat for other RNN models by changing the network and its associated hyperparameters